# Chapter 3.3

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics
import time

## Self-created functions

In [ ]:
def node_pruning(model, tester_model, x, y, layer_sizes, tol)

    loss, acc = model.evaluate(x, y, verbose=0, batch_size=512)
    original = model.get_weights()
    bas = [acc]
    bls = [loss]
    best_weights = model.get_weights()
    best_acc = 0
    best_loss = 1e20
    ol = loss
    oa = acc
    amounts = []
    places = []

    for layer, size in enumerate(layer_sizes):
        end_not_reached = True
        current_pos = 0
        num_removed = 0
        best_pos = -1
        best_change = tol
        nodes_removed = []
        print(f'Considering layer {len(layer_sizes) - layer}')
        while end_not_reached or improved:
            if not(end_not_reached):
                end_not_reached = True
                improved = False
                current_pos = 0
                size -= 1
                layer_sizes[layer] -= 1
                nodes_removed += [best_pos]
                best_weights[0][...,best_pos] = 0
                best_weights[1][best_pos] = 0
                best_pos = -1
                ol = best_loss
                oa = best_acc
                bas += [best_acc]
                bls += [best_loss]
                best_change = tol
                num_removed += 1
            if current_pos in nodes_removed:
                current_pos += 1
                if current_pos - num_removed >= size:
                    end_not_reached = False
                continue
            w = copy.deepcopy(best_weights)
            w[weight_len - (2*layer+1)][...,current_pos] = 0
            w[weight_len - 2*layer][current_pos] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=1024)
            # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
            if 0.3*(na - oa) + 0.7*(ol - nl) >= best_change:
                best_change = 0.3*(na - oa) + 0.7*(ol - nl)
                best_pos = current_pos
                improved = True
                best_acc = na
                best_loss = nl
            current_pos += 1
            if current_pos - num_removed >= size:
                print("Layer optimized")
                end_not_reached = False
        amounts.append(num_removed)
        places.append(nodes_removed)

    return best_weights, bas, bls, amounts, places

## Single-layer ANN

### MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 20
tol = 1e-5
layer_sizes = [128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem)
NR.to_csv('../../../results/num_removed_ANN_exh_pru_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_mnist.csv')
L.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [128]
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_mnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 20
tol = 1e-5
layer_sizes = [128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem)
NR.to_csv('../../../results/num_removed_ANN_exh_pru_val_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_val_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_val_mnist.csv')
L.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_val_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_val_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [128]
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_val_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_val_mnist.png")

### Fashion MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 20
tol = 1e-5
layer_sizes = [128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem)
NR.to_csv('../../../results/num_removed_ANN_exh_pru_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_fmnist.csv')
L.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [128]
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_fmnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 20
tol = 1e-5
layer_sizes = [128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem)
NR.to_csv('../../../results/num_removed_ANN_exh_pru_val_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_val_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_val_fmnist.csv')
L.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [128]
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_val_fmnist.png")

## Multi-layer Perceptron

### MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 10
tol = 1e-5
layer_sizes = [32, 64, 128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_MLP_exh_pru_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_MLP_exh_pru_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_MLP_exh_pru_mnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_MLP_exh_pru_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_MLP_exh_pru_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_exh_pru_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_exh_pru_mnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 10
tol = 1e-5
layer_sizes = [32, 64, 128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_MLP_exh_pru_val_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_MLP_exh_pru_val_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_MLP_exh_pru_val_mnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_MLP_exh_pru_val_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_MLP_exh_pru_val_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_exh_pru_val_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_exh_pru_val_mnist.png")

### Fashion MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 10
tol = 1e-5
layer_sizes = [32, 64, 128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_MLP_exh_pru_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_MLP_exh_pru_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_MLP_exh_pru_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_MLP_exh_pru_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_MLP_exh_pru_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_exh_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_exh_pru_fmnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 10
tol = 1e-5
layer_sizes = [32, 64, 128]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_MLP_exh_pru_val_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_MLP_exh_pru_val_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_MLP_exh_pru_val_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_MLP_exh_pru_val_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_MLP_exh_pru_val_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_exh_pru_val_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_exh_pru_val_fmnist.png")

## Convolutional Neural Network

### MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_mnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_mnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_val_mnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_val_mnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_val_mnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_val_mnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_val_mnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_val_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_val_mnist.png")

### Fashion MNIST

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_fmnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_val_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_val_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_val_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_val_fmnist.png")

### CIFAR10

#### Set seed

In [3]:
np.random.seed(2020)

In [5]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_fmnist.png")

### Validation removal

In [3]:
np.random.seed(2020)

In [5]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [7]:
trials = 5
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem = []
delta_accs = []
delta_losses = []
for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
    num_rem += tmp_a
    a, l = model.evaluate(x_test, y_test, verbose=0)
    tester_model.set_weights(shrinked_weights)
    na, nl = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs += [na - a]
    delta_losses += [nl - l]

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [ ]:
NR = pd.Series(num_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
NR.to_csv('../../../results/num_removed_ANN_exh_pru_val_fmnist.csv')
NR.describe()

In [ ]:
A = pd.Series(delta_accs)
A.to_csv('../../../results/change_accuracy_ANN_exh_pru_val_fmnist.csv')
A.describe()

In [ ]:
L = pd.Series(delta_losses)
L.to_csv('../../../results/chnage_loss_ANN_exh_pru_val_fmnist.csv')
L.describe()

In [ ]:
num_rem = [sum(rem) for rem in num_rem]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=num_rem, y=delta_accs, marker='o', color='red', s=60)
plt.title("Change in accuracy depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_accuracy_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.scatter(x=num_rem, y=delta_losses, marker='o', color='red', s=60)
plt.title("Change in loss depending on number of nodes removed", size=28)
plt.xlabel("Number of nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/change_loss_vs_nodes_removed_ANN_exh_pru_val_fmnist.png")

In [ ]:
tol = 1e-5
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_train, y_train, layer_sizes, tol)
a, l = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
na, nl = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_ANN_exh_pru_val_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layer-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_ANN_exh_pru_val_fmnist.png")